In [ ]:
import splink
import pandas as pd
from splink import DuckDBAPI, Linker, SettingsCreator, block_on
import splink.comparison_library as cl
import psycopg2
import splink.comparison_library as cl
from splink.backends.duckdb import DuckDBAPI
import pandas as pd
import splink.comparison_library as cl
from splink import Linker, SettingsCreator, block_on, splink_datasets
from splink.backends.duckdb import DuckDBAPI
import splink.comparison_level_library as cll 


from credentials import host,port,db,user,password



In [3]:

conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=db,
    user=user,
    password=password
)

query = """SELECT * FROM all_tenants               
  """


In [4]:
df=pd.read_sql(query, conn)
df=df.rename(columns={'source_table':'source_dataset'})
 

C:\Users\jolya\AppData\Local\Temp\ipykernel_8156\1035853623.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query, conn)


In [5]:
df.sample(5)

,id,company_name,website,company_phone,alternative_name,source_property_id,employee_total,year_established,naics,naics_description,sic,sic_description,sales_volume,type_of_location,state,city,zip_code,street_address,source_dataset
315370,dbusa|548738239eb5fe93dd78d1bcce1f20ce,porter ranch plaza clinic,www.facey.com,8184032400,None,11140004107067,NaN,2015.0,621111,offices of physicians (except mental health sp...,8011219,pediatrics,2271000.0,None,ca,porter ranch,91326,19950 rinaldi st,dbusa
37615,reonomy|b3a434c37f857e32da5081ce79c8b786,anthony e good,None,3047231130,None,db611f80-cdcc-5572-9f44-01e7e02bd6f2,1.0,2012.0,621399,offices of all other miscellaneous health prac...,80490000,offices of health practitioner,32881.0,single location,wv,weirton,26062,485 colliers way ste a,reonomy
438941,dbusa|74212485f5033085ac81be649b219330,providence heart,www.scheart.com,8037943950,None,11132546532021,NaN,1988.0,621111,offices of physicians (except mental health sp...,8011115,cardiovascular disease,3120000.0,None,sc,lexington,29072,108 palmetto park blvd ste c,dbusa
456030,dbusa|ab0561a5a31d6e701d00e70a0194adbb,mimbres memorial hospital and nursing home,www.mimbresmemorial.com,5755465800,None,11132545477362,NaN,1996.0,621391,offices of podiatrists,8011107,allergy & immunology,7666000.0,None,nm,deming,88030,900 w ash st,dbusa
219884,reonomy|6f19b0069383f9b743a99cb2729dfa6f,"christiana care health system, inc., dba asari...",www.christianacare.org,3026237500,"asari, julie y md",2fc08e64-56d6-59d4-8174-7fee182b6eef,0.0,NaN,622110,general medical and surgical hospitals,80620000,general medical and surgical hospitals,0.0,branch,de,wilmington,19808,4512 kirkwood hwy ste 300,reonomy


In [6]:
from splink.blocking_analysis import cumulative_comparisons_to_be_scored_from_blocking_rules_chart
db_api = DuckDBAPI()



prediction_rules = [
     block_on("street_address"),
     block_on("company_name"),
     block_on("website"),
    block_on("company_name","website"),


    block_on("zip_code"),
    block_on("city","state"),
    block_on("city","naics","sic")]
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=df,
    blocking_rules=prediction_rules,
    db_api=db_api,
    link_type="link_and_dedupe",
    source_dataset_column_name='source_dataset',
    unique_id_column_name='id'
)

alt.Chart(...)

In [7]:
naics_comparison = cl.CustomComparison(
    output_column_name="naics",
    comparison_levels=[
        cll.NullLevel("naics"),
        
        cll.ExactMatchLevel("naics"),
        
        cll.CustomLevel(
            sql_condition="SUBSTR(naics_l::text, 1, 4) = SUBSTR(naics_r::text, 1, 4)",
            label_for_charts="4-digit NAICS match"
        ),
        
        cll.CustomLevel(
            sql_condition="SUBSTR(naics_l::text, 1, 2) = SUBSTR(naics_r::text, 1, 2)",
            label_for_charts="2-digit NAICS match"
        ),

        cll.ElseLevel()
    ]
)




sic_comparison = cl.CustomComparison(
    output_column_name="sic",
    comparison_levels=[
        cll.NullLevel("sic"),
        
        cll.ExactMatchLevel("sic"),
        
        cll.CustomLevel(
            sql_condition="SUBSTR(sic_l::text, 1, 2) = SUBSTR(sic_r::text, 1, 2)",
            label_for_charts="2-digit SIC match"
        ),
        cll.ElseLevel()
    ]
)


sales_volume_comparison = cl.CustomComparison(
    output_column_name="sales_volume",
    comparison_levels=[
        cll.NullLevel("sales_volume"),
        cll.ExactMatchLevel("sales_volume"),
        cll.PercentageDifferenceLevel("sales_volume", percentage_threshold=0.01),
        cll.PercentageDifferenceLevel("sales_volume", percentage_threshold=0.05),
        cll.ElseLevel()
    ]
)






settings=SettingsCreator( link_type='link_and_dedupe',
                         comparisons=[
                             
                             cl.NameComparison("company_name"),
                             cl.ExactMatch("state"),
                             cl.ExactMatch("city"),
                             cl.ExactMatch("year_established"),
                             cl.ExactMatch("website"),
                             cl.ExactMatch("type_of_location"),
                             
                             
                             
                             cl.LevenshteinAtThresholds("street_address",[1,3,4]),
                             cl.ExactMatch("zip_code"),
                             cl.LevenshteinAtThresholds("company_phone",[1,2]),
                             naics_comparison,
                             sic_comparison,
                             sales_volume_comparison
                             
                             
                             
                             
                         ],
                         
                         
                         blocking_rules_to_generate_predictions=[
                             block_on("website"),block_on("company_name"),block_on("street_address","city","zip_code"),block_on("company_name","city","zip_code"),block_on("zip_code","naics","sic")
                                                                ],
                         unique_id_column_name="id",
                         additional_columns_to_retain=["company_name","alternative_name","website","zip_code","street_address","year_established","employee_total","naics","sic","company_phone","sales_volume"]
                         
    
                        )

api=DuckDBAPI()
linker=Linker(df,settings,api)
linker.training.estimate_u_using_random_sampling(max_pairs=1e7)
linker.training.estimate_parameters_using_expectation_maximisation(block_on("street_address","zip_code"))
#linker.training.estimate_parameters_using_expectation_maximisation(block_on("zip_code"))
linker.training.estimate_parameters_using_expectation_maximisation(block_on("company_name","naics","sic"))
linker.training.estimate_parameters_using_expectation_maximisation(block_on("company_name","website"))
linker.training.estimate_parameters_using_expectation_maximisation(block_on("zip_code","naics","sic"))





----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - company_name (no m values are trained).
    - state (no m values are trained).
    - city (no m values are trained).
    - year_established (no m values are trained).
    - website (no m values are trained).
    - type_of_location (no m values are trained).
    - street_address (no m values are trained).
    - zip_code (no m values are trained).
    - company_phone (no m values are trained).
    - naics (no m values are trained).
    - sic (no m values are trained).
    - sales_volume (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."street_address" = r."street_address") AND (l."zip_code" = r."zip_code")

Parameter estimates will be made for the following comparison(s):
    - company_name
    - state
    - city
    - year_establi

<EMTrainingSession, blocking on (l."zip_code" = r."zip_code") AND (l."naics" = r."naics") AND (l."sic" = r."sic"), deactivating comparisons zip_code, naics, sic>

In [8]:
df_predictions = linker.inference.predict(threshold_match_probability=0.97)
preds=df_predictions.as_pandas_dataframe(limit=100000)
preds



Blocking time: 20.93 seconds
Predict time: 234.25 seconds


,match_weight,match_probability,source_dataset_l,source_dataset_r,id_l,id_r,company_name_l,company_name_r,gamma_company_name,state_l,...,sic_r,gamma_sic,sales_volume_l,sales_volume_r,gamma_sales_volume,alternative_name_l,alternative_name_r,employee_total_l,employee_total_r,match_key
0,12.639768,0.999843,dbusa,dbusa,dbusa|b84a6677b029f5f935ed141cdf290b48,dbusa|ce40f981a3ef8dcdc7c93eaf7109bd7e,davita,davita,4,wi,...,8099001,2,1494000.0,1910000.0,0,None,davita-clarkston dialysis,69000.0,69000.0,0
1,9.432973,0.998555,dbusa,dbusa,dbusa|32eea76852bcb6a17ba791aab818289b,dbusa|ce40f981a3ef8dcdc7c93eaf7109bd7e,davita,davita,4,wi,...,8099001,1,1494000.0,1910000.0,0,None,davita-clarkston dialysis,69000.0,69000.0,0
2,6.232982,0.986880,dbusa,dbusa,dbusa|25bbc647e69c37c9d170513e9c760b2d,dbusa|b67097133bf574f831fdf0808c779a95,chase,chase,4,oh,...,6153009,1,1819000.0,2384000.0,0,None,None,201005.0,201005.0,0
3,6.232982,0.986880,dbusa,dbusa,dbusa|693b1f0a82de4fe1ca10a23fadea8696,dbusa|b67097133bf574f831fdf0808c779a95,chase,chase,4,oh,...,6153009,1,1819000.0,2384000.0,0,None,None,201005.0,201005.0,0
4,6.232982,0.986880,dbusa,dbusa,dbusa|56949149a68c692edb84dccbc446b6bd,dbusa|b67097133bf574f831fdf0808c779a95,chase,chase,4,oh,...,6153009,1,1819000.0,2384000.0,0,None,None,201005.0,201005.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,57.205613,1.000000,dbusa,dbusa,dbusa|08b5d00dafa2bb5cf02b1e31118ae3be,dbusa|50e41e464b8c04c2d990e344e9619262,bankers life & casualty company,bankers life & casualty company,4,il,...,6411014,0,300700000.0,300700000.0,3,None,None,7677.0,7677.0,0
99996,57.205613,1.000000,dbusa,dbusa,dbusa|041f17f6ab37baec53cfd4ea884528c4,dbusa|50e41e464b8c04c2d990e344e9619262,bankers life & casualty company,bankers life & casualty company,4,il,...,6411014,0,300700000.0,300700000.0,3,None,None,7677.0,7677.0,0
99997,60.194886,1.000000,dbusa,dbusa,dbusa|3ec6ae796008e4781290ca2499262289,dbusa|50e41e464b8c04c2d990e344e9619262,bankers life & casualty company,bankers life & casualty company,4,il,...,6411014,0,300700000.0,300700000.0,3,None,None,7677.0,7677.0,0
99998,60.194886,1.000000,dbusa,dbusa,dbusa|1902e02400b7b5404770a614fb50311c,dbusa|50e41e464b8c04c2d990e344e9619262,bankers life & casualty company,bankers life & casualty company,4,il,...,6411014,0,300700000.0,300700000.0,3,None,None,7677.0,7677.0,0


In [9]:
df_clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    df_predictions, threshold_match_probability=0.97
)
df_clusters=df_clusters.as_pandas_dataframe()
result=df_clusters[['cluster_id','id']]


Completed iteration 1, num representatives needing updating: 190596
Completed iteration 2, num representatives needing updating: 170502
Completed iteration 3, num representatives needing updating: 179269
Completed iteration 4, num representatives needing updating: 170062
Completed iteration 5, num representatives needing updating: 168505
Completed iteration 6, num representatives needing updating: 151445
Completed iteration 7, num representatives needing updating: 137245
Completed iteration 8, num representatives needing updating: 124817
Completed iteration 9, num representatives needing updating: 108477
Completed iteration 10, num representatives needing updating: 106083
Completed iteration 11, num representatives needing updating: 88711
Completed iteration 12, num representatives needing updating: 80615
Completed iteration 13, num representatives needing updating: 58377
Completed iteration 14, num representatives needing updating: 46664
Completed iteration 15, num representatives nee

In [10]:
import io
with conn.cursor() as curs:
    curs.execute("DROP TABLE IF EXISTS matched_tenants")  
    curs.execute("CREATE TABLE IF NOT EXISTS matched_tenants( cluster_id text, id text)")
    buffer = io.StringIO()
    
    result.to_csv(buffer, index=False, header=False)
    
    buffer.seek(0)

    copy_sql = f"""
        COPY matched_tenants ("cluster_id","id") 
        FROM STDIN WITH (FORMAT CSV)
    """

    curs.copy_expert(sql=copy_sql, file=buffer)
    conn.commit()
    print("Copy complete.")

    

Copy complete.
